In [25]:
import os
import sys
from pathlib import Path
import cv2
import numpy as np
import re
from typing import Dict, Tuple, Optional


def setup_project_path():
    current = Path.cwd()
    while not (current / 'craft').exists():
        current = current.parent
    return current
project_root = setup_project_path()
sys.path.insert(0, str(project_root))

In [26]:
def imwrite_unicode(path, img):
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True


In [27]:
def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    try:
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None


In [28]:
def load_centers_from_file(centers_file: str):
    centers = []
    with open(centers_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            x_str, y_str = line.split(",")
            centers.append((float(x_str), float(y_str)))
    return centers

def load_tilts_from_file(tilt_file: str):
    tilts = []
    with open(tilt_file, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            tilts.append(float(s))
    return tilts

def load_spacings_from_file(spacing_file: str):
    items = []
    with open(spacing_file, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            parts = s.split(",")
            if len(parts) != 4:
                raise ValueError(f"Invalid spacing line: {s}")
            i1 = int(parts[0])
            i2 = int(parts[1])
            spacing = float(parts[2])
            flag = parts[3].strip()
            items.append({"i1": i1, "i2": i2, "spacing": spacing, "flag": flag})
    return items


In [29]:
def draw_label(img, text, x, y, scale=0.3, thickness=1, pad=3):
    text = str(text)
    (tw, th), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale, thickness)
    x, y = int(x), int(y)

    x = max(0, min(x, img.shape[1] - 1))
    y = max(th + pad, min(y, img.shape[0] - 1))

    cv2.rectangle(img, (x, y - th - pad), (x + tw + pad * 2, y + baseline + pad), (0, 0, 0), -1)
    cv2.putText(img, text, (x + pad, y), cv2.FONT_HERSHEY_SIMPLEX, scale, (255, 255, 255),
                thickness, cv2.LINE_AA)

In [30]:
def _put_text(
    img, text, x, y,
    scale=0.6, thickness=2,
    fg_color=(255, 255, 255), bg_color=(0, 0, 0),
    pad=3,
    anchor="lt",
):
    text = str(text)
    (tw, th), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale, thickness)
    x, y = float(x), float(y)

    if anchor == "center":
        x = x - (tw + pad * 2) / 2.0
        y = y + (th + pad) / 2.0

    x, y = int(x), int(y)

    cv2.rectangle(
        img,
        (x, y - th - pad),
        (x + tw + pad * 2, y + baseline + pad),
        bg_color,
        -1,
    )

    cv2.putText(
        img,
        text,
        (x + pad, y),
        cv2.FONT_HERSHEY_SIMPLEX,
        scale,
        fg_color,
        thickness,
        cv2.LINE_AA,
    )


In [31]:
def visualize(
    image_path: str,
    centers_file: str,
    out_path: str,
    tilt_file: str | None = None,
    spacing_file: str | None = None,
    *,
    show_center: bool = True,
    show_tilt: bool = True,
    show_space: bool = True,
    show_center_coords: bool = True,
    sort_by_x: bool = True,
    draw_pair_lines: bool = True,
    center_radius: int = 4,
):
    img = imread_unicode(image_path)
    if img is None:
        raise FileNotFoundError(f"Cannot read image: {image_path}")

    centers = load_centers_from_file(centers_file)
    if sort_by_x:
        centers = sorted(centers, key=lambda p: p[0])

    if show_center:
        for (cx, cy) in centers:
            cv2.circle(img, (int(cx), int(cy)), center_radius, (0, 255, 0), -1)
            cv2.circle(img, (int(cx), int(cy)), center_radius + 2, (0, 0, 0), 2)
            if show_center_coords:
                draw_label(img, f"({cx:.1f},{cy:.1f})", cx + 6, cy - 6, scale=0.45, thickness=1)

    if show_tilt and tilt_file and os.path.exists(tilt_file):
        tilts = load_tilts_from_file(tilt_file)
        pair_count = max(0, len(centers) - 1)
        m = min(pair_count, len(tilts))

        for i in range(m):
            (x1, y1) = centers[i]
            (x2, y2) = centers[i + 1]

            if draw_pair_lines:
                cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

            mx = (x1 + x2) / 2.0
            my = (y1 + y2) / 2.0

            _put_text(
                img,
                f"{tilts[i]:.2f}",
                mx,
                my - 6,
                scale=0.3,
                thickness=1,
                fg_color=(255, 255, 255),
                bg_color=(0, 0, 0),
                anchor="center",
            )

    if show_space and spacing_file and os.path.exists(spacing_file):
        items = load_spacings_from_file(spacing_file)

        for it in items:
            i1 = it["i1"]
            i2 = it["i2"]
            spacing = it["spacing"]
            flag = it["flag"]

            if i1 < 0 or i2 >= len(centers) or i2 != i1 + 1:
                continue

            (x1, y1) = centers[i1]
            (x2, y2) = centers[i2]

            if draw_pair_lines:
                color = (0, 165, 255) if flag == "gap" else (255, 0, 255)
                cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

            mx = (x1 + x2) / 2.0
            my = (y1 + y2) / 2.0

            label = f"{spacing:.1f}"
            if flag == "gap":
                label += " gap"

            _put_text(
                img,
                label,
                mx,
                my + 8,
                scale=0.3,
                thickness=1,
                fg_color=(255, 255, 255),
                bg_color=(0, 0, 0),
                anchor="center",
            )

    imwrite_unicode(out_path, img)


In [32]:
def score_to_heatmap(score: np.ndarray, colormap=cv2.COLORMAP_JET):
    s = score.astype(np.float32)
    mn, mx = float(np.min(s)), float(np.max(s))
    if mx - mn < 1e-8:
        norm = np.zeros_like(s, dtype=np.uint8)
    else:
        norm = ((s - mn) / (mx - mn) * 255.0).astype(np.uint8)
    heat = cv2.applyColorMap(norm, colormap)  # BGR
    return heat


In [33]:
def overlay_heatmap_on_image(img_bgr: np.ndarray, heat_bgr: np.ndarray, alpha=0.45):
    if img_bgr.shape[:2] != heat_bgr.shape[:2]:
        heat_bgr = cv2.resize(heat_bgr, (img_bgr.shape[1], img_bgr.shape[0]), interpolation=cv2.INTER_LINEAR)
    return cv2.addWeighted(img_bgr, 1.0 - alpha, heat_bgr, alpha, 0)


In [34]:
def _find_image_path(image_folder: str, name: str):
    for ext in [".jpg", ".jpeg", ".png", ".bmp", ".webp"]:
        cand = os.path.join(image_folder, name + ext)
        if os.path.exists(cand):
            return cand
    return None


In [35]:
def run_all_visualizations(
    *,
    image_folder: str,
    centers_folder: str,
    out_root: str,
    tilts_folder: str | None = None,
    spacing_folder: str | None = None,
    score_text_folder: str | None = None,
    alpha: float = 0.45,
    sort_by_x: bool = True,
    draw_pair_lines: bool = True,
    save_score_heatmap: bool = True, 
):
    os.makedirs(out_root, exist_ok=True)

    center_txts = [f for f in os.listdir(centers_folder) if f.startswith("res_") and f.endswith("_center.txt")]
    if not center_txts:
        print(f"[WARN] No center txt found in: {centers_folder}")
        return

    for fname in sorted(center_txts):
        name = fname[len("res_") : -len("_center.txt")]
        centers_file = os.path.join(centers_folder, fname)

        img_path = _find_image_path(image_folder, name)
        if img_path is None:
            print(f"[WARN] image not found for '{name}' in {image_folder}")
            continue

        tilt_file = os.path.join(tilts_folder, f"res_{name}_tilt.txt") if tilts_folder else None
        spacing_file = os.path.join(spacing_folder, f"res_{name}_spacing.txt") if spacing_folder else None
        score_npy = os.path.join(score_text_folder, f"res_{name}_score_text.npy") if score_text_folder else None

        out_dir = os.path.join(out_root, name)
        os.makedirs(out_dir, exist_ok=True)

        try:
            visualize(
                image_path=img_path,
                centers_file=centers_file,
                tilt_file=tilt_file,
                spacing_file=spacing_file,
                out_path=os.path.join(out_dir, "vis_center.jpg"),
                show_center=True, show_tilt=False, show_space=False,
                show_center_coords=True,
                sort_by_x=sort_by_x,
                draw_pair_lines=draw_pair_lines,
            )
        except Exception as e:
            print(f"[WARN] center visualize failed '{name}': {e}")

        try:
            visualize(
                image_path=img_path,
                centers_file=centers_file,
                tilt_file=tilt_file,
                spacing_file=spacing_file,
                out_path=os.path.join(out_dir, "vis_center_tilt.jpg"),
                show_center=True, show_tilt=True, show_space=False,
                show_center_coords=False,
                sort_by_x=sort_by_x,
                draw_pair_lines=draw_pair_lines,
            )
        except Exception as e:
            print(f"[WARN] center+tilt visualize failed '{name}': {e}")

        try:
            visualize(
                image_path=img_path,
                centers_file=centers_file,
                tilt_file=tilt_file,
                spacing_file=spacing_file,
                out_path=os.path.join(out_dir, "vis_center_space.jpg"),
                show_center=True, show_tilt=False, show_space=True,
                show_center_coords=False,
                sort_by_x=sort_by_x,
                draw_pair_lines=draw_pair_lines,
            )
        except Exception as e:
            print(f"[WARN] center+space visualize failed '{name}': {e}")

        try:
            if score_npy and os.path.exists(score_npy):
                score = np.load(score_npy)
                heat = score_to_heatmap(score)

                if save_score_heatmap:
                    imwrite_unicode(os.path.join(out_dir, "vis_score_heat.jpg"), heat)

                img = imread_unicode(img_path)
                if img is None:
                    raise FileNotFoundError(f"Cannot read image for overlay: {img_path}")

                over = overlay_heatmap_on_image(img, heat, alpha=alpha)
                imwrite_unicode(os.path.join(out_dir, "vis_score_overlay.jpg"), over)
            else:
                print(f"[WARN] score_text npy not found for '{name}': {score_npy}")
        except Exception as e:
            print(f"[WARN] score overlay failed '{name}': {e}")

        print(f"[INFO] done: {name} -> {out_dir}")


In [36]:
run_all_visualizations(
    image_folder=str(project_root / "images"),
    centers_folder=str(project_root / "results" / "center"),
    tilts_folder=str(project_root / "results" / "tilt"),
    spacing_folder=str(project_root / "results" / "space"),
    score_text_folder=str(project_root / "results" / "score_text"),
    out_root=str(project_root / "visualize"/ "visualize_1"),
    alpha=0.45,
    save_score_heatmap=True
)


[INFO] done: test1 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test1
[INFO] done: test2 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test2
[INFO] done: test3 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test3
[INFO] done: test4 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test4
[INFO] done: test5 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test5
[INFO] done: test6 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test6
[INFO] done: test7 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test7
[INFO] done: test8 -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\visualize\visualize_1\test8
